In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install verstack


In [ ]:
!git clone https://github.com/analokmaus/kuma_utils.git

Cloning into 'kuma_utils'...
remote: Enumerating objects: 915, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 915 (delta 104), reused 102 (delta 96), pack-reused 795
Receiving objects: 100% (915/915), 679.99 KiB | 5.96 MiB/s, done.
Resolving deltas: 100% (592/592), done.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer#填补缺失的类
from sklearn.ensemble import RandomForestRegressor
import missingno as msno
import xgboost as xgb
# from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense ,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization

# data1



In [3]:
train = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data1/train.csv', index_col = None, header = None)
test = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data1/test.csv', index_col = None, header = None)
data = pd.concat([train, test], axis = 0)
col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
data.columns = col_name

In [4]:
print(train.shape)
print(test.shape)

(765, 9)
(309, 9)


In [ ]:
# 1-train.isna().sum().sum()/1074/9

0.8095385888681979

In [ ]:
data

,c0,c1,c2,c3,c4,c5,c6,c7,c8
0,NaN,0.0,0.0,NaN,2.5,1055.0,676.0,28.0,61.89
1,NaN,142.5,0.0,NaN,0.0,932.0,594.0,270.0,40.27
2,332.5,142.5,0.0,NaN,0.0,932.0,594.0,365.0,41.05
3,NaN,132.4,0.0,NaN,0.0,978.4,825.5,360.0,44.30
4,NaN,95.0,NaN,NaN,0.0,NaN,594.0,365.0,43.70
...,...,...,...,...,...,...,...,...,...
304,332.5,142.5,0.0,228.0,0.0,932.0,594.0,180.0,0.00
305,276.0,116.0,90.0,180.0,9.0,870.0,768.0,28.0,0.00
306,NaN,0.0,0.0,NaN,0.0,1125.0,613.0,180.0,0.00
307,116.0,173.0,NaN,192.0,NaN,NaN,NaN,NaN,0.00


In [ ]:
data_all = data.copy()
data_x = data_all.drop(['c8'], axis = 1)
data_y = data_all['c8'] #predict
data_y = pd.DataFrame(data_y)

In [ ]:
# from verstack import NaNImputer
# imputer = NaNImputer()
# data_x_impute = imputer.impute(data_x)

In [ ]:
# data_x_impute = pd.DataFrame(data_x_impute)
# data_x_impute

In [ ]:
from kuma_utils.preprocessing.imputer import LGBMImputer
lgbm_imtr = LGBMImputer(n_iter=500)
data_x_impute = lgbm_imtr.fit_transform(data_x)
data_x_impute = pd.DataFrame(data_x_impute)
data_x_impute

  0%|          | 0/8 [00:00<?, ?it/s]

,c0,c1,c2,c3,c4,c5,c6,c7
0,406.601627,0.0,0.000000,190.840409,2.500000,1055.000000,676.000000,28.000000
1,332.845514,142.5,0.000000,234.627601,0.000000,932.000000,594.000000,270.000000
2,332.500000,142.5,0.000000,227.931764,0.000000,932.000000,594.000000,365.000000
3,153.112983,132.4,0.000000,197.256562,0.000000,978.400000,825.500000,360.000000
4,387.351491,95.0,-4.417599,228.935905,0.000000,965.886745,594.000000,365.000000
...,...,...,...,...,...,...,...,...
304,332.500000,142.5,0.000000,228.000000,0.000000,932.000000,594.000000,180.000000
305,276.000000,116.0,90.000000,180.000000,9.000000,870.000000,768.000000,28.000000
306,497.094670,0.0,0.000000,192.020001,0.000000,1125.000000,613.000000,180.000000
307,116.000000,173.0,54.528441,192.000000,0.535524,929.891306,787.578204,31.200269


In [ ]:
X_train = data_x_impute.iloc[:train.shape[0]]
X_test = data_x_impute.iloc[-test.shape[0]:]
Y_train = data_y.iloc[:train.shape[0]]
Y_test = data_y.iloc[-test.shape[0]:]
scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(24, input_shape=(8,), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation=None))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)
ypred = model.predict(X_test)
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload1_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload1_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data1/upload1_12.csv',index = False, header = False)

In [ ]:
model.fit(train_x, Y_train, epochs=200, batch_size=10)

In [ ]:
model.fit(X_train, Y_train, epochs=200, batch_size=10)

In [ ]:
ypred = model.predict(X_test)

10/10 [==============================] - 0s 2ms/step


In [ ]:
ypred = model.predict(test_x)

10/10 [==============================] - 0s 2ms/step


In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload1_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload1_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data1/upload1_12.csv',index = False, header = False)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, gamma=0.1) 
print(xgbr)
xgbr.fit(X_train, Y_train)
ypred = xgbr.predict(X_test)

XGBRegressor(gamma=0.1, max_depth=4, min_child_weight=5, n_estimators=550,
             verbosity=0)


In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=3000,
    objective='regression', 
    metric='mae'
)


print(model)
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

LGBMRegressor(learning_rate=0.01, metric='mae', n_estimators=3000,
              objective='regression')


In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload1_9 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)

In [ ]:
upload1_9.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data1/upload1_9.csv',index = False, header = False)

# data2

In [5]:
train = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data2/train.csv', index_col = None, header = None)
test = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data2/test.csv', index_col = None, header = None)
data = pd.concat([train, test], axis = 0)
col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
data.columns = col_name

In [6]:
print(train.shape)
print(test.shape)

(574, 9)
(230, 9)


In [ ]:
data

,c0,c1,c2,c3,c4,c5,c6,c7,c8
0,NaN,514.5,NaN,NaN,7.0,2.0,0.0,0.0,15.55
1,NaN,NaN,318.5,122.50,7.0,2.0,0.0,0.0,20.84
2,0.90,NaN,318.5,122.50,7.0,NaN,0.0,NaN,21.46
3,0.90,563.5,318.5,122.50,7.0,4.0,0.0,0.0,20.71
4,0.90,563.5,318.5,122.50,7.0,5.0,NaN,0.0,19.68
...,...,...,...,...,...,...,...,...,...
225,0.79,637.0,343.0,147.00,NaN,5.0,NaN,NaN,0.00
226,0.64,784.0,343.0,NaN,3.5,5.0,0.4,5.0,0.00
227,0.90,NaN,318.5,122.50,NaN,4.0,0.1,NaN,0.00
228,NaN,563.5,318.5,122.50,7.0,2.0,NaN,4.0,0.00


In [ ]:
data_all = data.copy()
data_x = data_all.drop(['c8'], axis = 1)
data_y = data_all['c8'] #predict
data_y = pd.DataFrame(data_y)

In [ ]:
# from verstack import NaNImputer
# imputer = NaNImputer()
# data_x_impute = imputer.impute(data_x)

In [ ]:
# data_x_impute = pd.DataFrame(data_x_impute)
# data_x_impute

In [ ]:
# from kuma_utils.preprocessing.imputer import LGBMImputer
lgbm_imtr = LGBMImputer(n_iter=500)
data_x_impute = lgbm_imtr.fit_transform(data_x)
data_x_impute = pd.DataFrame(data_x_impute)
data_x_impute

  0%|          | 0/8 [00:00<?, ?it/s]

,c0,c1,c2,c3,c4,c5,c6,c7
0,0.16802,514.500000,329.504048,111.949710,7.000000,2.000000,0.0,0.000000
1,0.16802,534.531268,318.500000,122.500000,7.000000,2.000000,0.0,0.000000
2,0.90000,561.005301,318.500000,122.500000,7.000000,3.756309,0.0,-1.469279
3,0.90000,563.500000,318.500000,122.500000,7.000000,4.000000,0.0,0.000000
4,0.90000,563.500000,318.500000,122.500000,7.000000,5.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...
225,0.79000,637.000000,343.000000,147.000000,7.020153,5.000000,0.0,2.385304
226,0.64000,784.000000,343.000000,219.200741,3.500000,5.000000,0.4,5.000000
227,0.90000,559.059785,318.500000,122.500000,7.179162,4.000000,0.1,2.809889
228,0.16802,563.500000,318.500000,122.500000,7.000000,2.000000,0.0,4.000000


In [ ]:
X_train = data_x_impute.iloc[:train.shape[0]]
X_test = data_x_impute.iloc[-test.shape[0]:]
Y_train = data_y.iloc[:train.shape[0]]
Y_test = data_y.iloc[-test.shape[0]:]
scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(24, input_shape=(8,), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation=None))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)
ypred = model.predict(X_test)
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload2_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload2_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data2/upload2_12.csv',index = False, header = False)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, gamma=0.1) 
print(xgbr)
xgbr.fit(X_train, Y_train)
ypred = xgbr.predict(X_test)

XGBRegressor(gamma=0.1, max_depth=4, min_child_weight=5, n_estimators=550,
             verbosity=0)


In [ ]:
# from lightgbm import LGBMRegressor

model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=3000,
    objective='regression', 
    metric='mae'
)


print(model)
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

LGBMRegressor(learning_rate=0.01, metric='mae', n_estimators=3000,
              objective='regression')


In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload2_9 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)

In [ ]:
upload2_9.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data2/upload2_9.csv',index = False, header = False)

# data3

In [7]:
train = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data3/train.csv', index_col = None, header = None)
test = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data3/test.csv', index_col = None, header = None)
data = pd.concat([train, test], axis = 0)
# col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
# data.columns = col_name

In [9]:
print(train.shape)
print(test.shape)

(371, 14)
(151, 14)


In [ ]:
data

In [ ]:
data_all = data.copy()
data_x = data_all.drop([13], axis = 1)
data_y = data_all[13] #predict
data_y = pd.DataFrame(data_y)

In [ ]:
# from kuma_utils.preprocessing.imputer import LGBMImputer
lgbm_imtr = LGBMImputer(n_iter=500)
data_x_impute = lgbm_imtr.fit_transform(data_x)
data_x_impute = pd.DataFrame(data_x_impute)
data_x_impute

In [ ]:
X_train = data_x_impute.iloc[:train.shape[0]]
X_test = data_x_impute.iloc[-test.shape[0]:]
Y_train = data_y.iloc[:train.shape[0]]
Y_test = data_y.iloc[-test.shape[0]:]
scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(24, input_shape=(13,), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation=None))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)
ypred = model.predict(X_test)
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload3_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload3_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data3/upload3_12.csv',index = False, header = False)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, gamma=0.1) 
print(xgbr)
xgbr.fit(X_train, Y_train)
ypred = xgbr.predict(X_test)

In [ ]:
# from lightgbm import LGBMRegressor

model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=3000,
    objective='regression', 
    metric='mae'
)


print(model)
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload3_9 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)

In [ ]:
upload3_9.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data3/upload3_9.csv',index = False, header = False)

# data4

In [10]:
train = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data4/train.csv', index_col = None, header = None)
test = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data4/test.csv', index_col = None, header = None)
data = pd.concat([train, test], axis = 0)
# col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
# data.columns = col_name

In [11]:
print(train.shape)
print(test.shape)

(6064, 9)
(2457, 9)


In [ ]:
data

,0,1,2,3,4,5,6,7,8
0,-0.0151,NaN,NaN,1.3097,0.9880,NaN,0.6641,NaN,0.5365
1,1.5632,NaN,NaN,NaN,1.1491,NaN,1.5630,-0.9377,0.5189
2,0.1995,NaN,-1.3563,-0.0805,-0.9766,0.8299,-0.8556,NaN,0.4942
3,1.5137,-0.2172,0.4891,0.5671,1.4003,0.3817,1.4960,-1.3281,0.2852
4,0.0454,1.2973,NaN,NaN,0.7891,NaN,-1.3414,0.1989,1.1178
...,...,...,...,...,...,...,...,...,...
2452,NaN,0.9956,-1.0918,NaN,-0.9488,NaN,0.2098,-0.7696,0.0000
2453,0.8109,0.0437,-0.0814,-0.0590,0.8420,-1.0750,-0.6685,-1.1301,0.0000
2454,0.3368,-0.1631,-1.1380,1.3471,-0.5409,-1.0533,1.3027,NaN,0.0000
2455,NaN,-0.5297,1.1033,NaN,-0.5938,NaN,NaN,-0.5765,0.0000


In [ ]:
data_all = data.copy()
data_x = data_all.drop([8], axis = 1)
data_y = data_all[8] #predict
data_y = pd.DataFrame(data_y)

In [ ]:
# from kuma_utils.preprocessing.imputer import LGBMImputer
lgbm_imtr = LGBMImputer(n_iter=500)
data_x_impute = lgbm_imtr.fit_transform(data_x)
data_x_impute = pd.DataFrame(data_x_impute)
data_x_impute

  0%|          | 0/8 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7
0,-0.015100,0.203242,0.085716,1.309700,0.9880,-0.029749,0.6641,0.155955
1,1.563200,0.091347,-0.328187,0.429260,1.1491,0.438023,1.5630,-0.937700
2,0.199500,0.154533,-1.356300,-0.080500,-0.9766,0.829900,-0.8556,0.283754
3,1.513700,-0.217200,0.489100,0.567100,1.4003,0.381700,1.4960,-1.328100
4,0.045400,1.297300,0.422277,0.089117,0.7891,-0.136217,-1.3414,0.198900
...,...,...,...,...,...,...,...,...
2452,0.328667,0.995600,-1.091800,0.310690,-0.9488,-0.313962,0.2098,-0.769600
2453,0.810900,0.043700,-0.081400,-0.059000,0.8420,-1.075000,-0.6685,-1.130100
2454,0.336800,-0.163100,-1.138000,1.347100,-0.5409,-1.053300,1.3027,0.339440
2455,-0.291751,-0.529700,1.103300,0.298801,-0.5938,-0.068735,0.2988,-0.576500


In [ ]:
X_train = data_x_impute.iloc[:train.shape[0]]
X_test = data_x_impute.iloc[-test.shape[0]:]
Y_train = data_y.iloc[:train.shape[0]]
Y_test = data_y.iloc[-test.shape[0]:]
scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(24, input_shape=(8,), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation=None))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)
ypred = model.predict(X_test)
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload4_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload4_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data4/upload4_12.csv',index = False, header = False)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, gamma=0.1) 
print(xgbr)
xgbr.fit(X_train, Y_train)
ypred = xgbr.predict(X_test)

XGBRegressor(gamma=0.1, max_depth=4, min_child_weight=5, n_estimators=550,
             verbosity=0)


In [ ]:
# from lightgbm import LGBMRegressor

model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=3000,
    objective='regression', 
    metric='mae'
)


print(model)
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

LGBMRegressor(learning_rate=0.01, metric='mae', n_estimators=3000,
              objective='regression')


In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload4_9 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)

In [ ]:
upload4_9.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data4/upload4_9.csv',index = False, header = False)

# data5


In [12]:
train = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data5/train.csv', index_col = None, header = None)
test = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data5/test.csv', index_col = None, header = None)
data = pd.concat([train, test], axis = 0)
# col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
# data.columns = col_name

In [13]:
print(train.shape)
print(test.shape)

(7077, 5)
(2870, 5)


In [ ]:
data

,0,1,2,3,4
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,NaN,56.90,1007.15,NaN,438.76
3,11.80,40.66,NaN,NaN,464.43
4,13.97,39.16,NaN,84.60,470.96
...,...,...,...,...,...
2865,NaN,41.35,1004.29,99.10,0.00
2866,9.47,41.40,NaN,87.90,0.00
2867,15.11,NaN,NaN,46.91,0.00
2868,7.46,41.26,NaN,96.51,0.00


In [ ]:
data_all = data.copy()
data_x = data_all.drop([4], axis = 1)
data_y = data_all[4] #predict
data_y = pd.DataFrame(data_y)

In [ ]:
# from kuma_utils.preprocessing.imputer import LGBMImputer
lgbm_imtr = LGBMImputer(n_iter=500)
data_x_impute = lgbm_imtr.fit_transform(data_x)
data_x_impute = pd.DataFrame(data_x_impute)
data_x_impute

  0%|          | 0/4 [00:00<?, ?it/s]

,0,1,2,3
0,8.340000,40.770000,1010.840000,90.010000
1,23.640000,58.490000,1011.400000,74.200000
2,24.293664,56.900000,1007.150000,79.577765
3,11.800000,40.660000,1017.871127,79.936910
4,13.970000,39.160000,1013.607014,84.600000
...,...,...,...,...
2865,10.676595,41.350000,1004.290000,99.100000
2866,9.470000,41.400000,1024.064041,87.900000
2867,15.110000,43.075773,1014.024917,46.910000
2868,7.460000,41.260000,1011.804348,96.510000


In [ ]:
X_train = data_x_impute.iloc[:train.shape[0]]
X_test = data_x_impute.iloc[-test.shape[0]:]
Y_train = data_y.iloc[:train.shape[0]]
Y_test = data_y.iloc[-test.shape[0]:]
scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(24, input_shape=(4,), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation=None))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)
ypred = model.predict(X_test)
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload5_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload5_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data5/upload5_12.csv',index = False, header = False)

90/90 [==============================] - 0s 2ms/step


In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, gamma=0.1) 
print(xgbr)
xgbr.fit(X_train, Y_train)
ypred = xgbr.predict(X_test)

XGBRegressor(gamma=0.1, max_depth=4, min_child_weight=5, n_estimators=550,
             verbosity=0)


In [ ]:
# from lightgbm import LGBMRegressor

model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=3000,
    objective='regression', 
    metric='mae'
)


print(model)
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

LGBMRegressor(learning_rate=0.01, metric='mae', n_estimators=3000,
              objective='regression')


In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload5_9 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)

In [ ]:
upload5_9.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data5/upload5_9.csv',index = False, header = False)

# data6

In [14]:
train = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data6/train.csv', index_col = None, header = None)
test = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data6/test.csv', index_col = None, header = None)
data = pd.concat([train, test], axis = 0)
# col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
# data.columns = col_name

In [15]:
print(train.shape)
print(test.shape)

(1187, 12)
(479, 12)


In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,NaN,0.00,NaN,0.076,11.0,34.0,0.9978,NaN,0.56,9.4,5.0
1,7.8,0.880,0.00,2.6,0.098,25.0,NaN,NaN,NaN,0.68,9.8,5.0
2,NaN,0.760,NaN,2.3,0.092,15.0,54.0,NaN,3.26,0.65,NaN,5.0
3,11.2,0.280,NaN,NaN,0.075,17.0,60.0,0.9980,NaN,0.58,NaN,6.0
4,7.4,0.700,NaN,1.9,0.076,11.0,NaN,0.9978,3.51,0.56,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
474,7.4,0.785,0.19,NaN,NaN,19.0,98.0,0.9971,NaN,NaN,9.6,0.0
475,7.8,0.530,0.04,1.7,0.076,17.0,31.0,0.9964,3.33,NaN,NaN,0.0
476,NaN,0.460,0.45,NaN,0.065,7.0,NaN,NaN,3.32,0.79,14.0,0.0
477,7.8,NaN,NaN,NaN,0.068,11.0,NaN,0.9960,3.34,0.61,11.6,0.0


In [ ]:
data_all = data.copy()
data_x = data_all.drop([11], axis = 1)
data_y = data_all[11] #predict
data_y = pd.DataFrame(data_y)

In [ ]:
# from kuma_utils.preprocessing.imputer import LGBMImputer
lgbm_imtr = LGBMImputer(n_iter=500)
data_x_impute = lgbm_imtr.fit_transform(data_x)
data_x_impute = pd.DataFrame(data_x_impute)
data_x_impute

  0%|          | 0/11 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,10
0,7.400000,0.067335,0.000000,2.037114,7.600000e-02,11.0,34.000000,0.997800,3.016295,0.560000,9.400000
1,7.800000,0.880000,0.000000,2.600000,9.800000e-02,25.0,56.756795,0.016927,3.013921,0.680000,9.800000
2,8.504156,0.760000,-0.000131,2.300000,9.200000e-02,15.0,54.000000,0.025964,3.260000,0.650000,9.699104
3,11.200000,0.280000,0.000067,3.631662,7.500000e-02,17.0,60.000000,0.998000,2.875564,0.580000,9.300022
4,7.400000,0.700000,-0.000006,1.900000,7.600000e-02,11.0,47.297223,0.997800,3.510000,0.560000,9.062912
...,...,...,...,...,...,...,...,...,...,...,...
474,7.400000,0.785000,0.190000,3.098315,-1.847744e-09,19.0,98.000000,0.997100,2.962992,-0.004998,9.600000
475,7.800000,0.530000,0.040000,1.700000,7.600000e-02,17.0,31.000000,0.996400,3.330000,0.007609,9.170326
476,8.577786,0.460000,0.450000,2.340102,6.500000e-02,7.0,20.899845,0.010395,3.320000,0.790000,14.000000
477,7.800000,0.013547,0.000005,1.418411,6.800000e-02,11.0,27.789401,0.996000,3.340000,0.610000,11.600000


In [ ]:
X_train = data_x_impute.iloc[:train.shape[0]]
X_test = data_x_impute.iloc[-test.shape[0]:]
Y_train = data_y.iloc[:train.shape[0]]
Y_test = data_y.iloc[-test.shape[0]:]
scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(24, input_shape=(11,), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation=None))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)
ypred = model.predict(X_test)
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload6_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload6_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data6/upload6_12.csv',index = False, header = False)

Epoch 1/200
119/119 [==============================] - 1s 2ms/step - loss: 1.8269
Epoch 2/200
119/119 [==============================] - 0s 2ms/step - loss: 0.7694
Epoch 3/200
119/119 [==============================] - 0s 2ms/step - loss: 0.7122
Epoch 4/200
119/119 [==============================] - 0s 2ms/step - loss: 0.6547
Epoch 5/200
119/119 [==============================] - 0s 2ms/step - loss: 0.6501
Epoch 6/200
119/119 [==============================] - 0s 2ms/step - loss: 0.6297
Epoch 7/200
119/119 [==============================] - 0s 2ms/step - loss: 0.6274
Epoch 8/200
119/119 [==============================] - 0s 2ms/step - loss: 0.6016
Epoch 9/200
119/119 [==============================] - 0s 2ms/step - loss: 0.5956
Epoch 10/200
119/119 [==============================] - 0s 2ms/step - loss: 0.5771
Epoch 11/200
119/119 [==============================] - 0s 2ms/step - loss: 0.5739
Epoch 12/200
119/119 [==============================] - 0s 2ms/step - loss: 0.5661
Epoch 13/200


In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, gamma=0.1) 
print(xgbr)
xgbr.fit(X_train, Y_train)
ypred = xgbr.predict(X_test)

XGBRegressor(gamma=0.1, max_depth=4, min_child_weight=5, n_estimators=550,
             verbosity=0)


In [ ]:
# from lightgbm import LGBMRegressor

model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=3000,
    objective='regression', 
    metric='mae'
)


print(model)
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

LGBMRegressor(learning_rate=0.01, metric='mae', n_estimators=3000,
              objective='regression')


In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload6_9 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)

In [ ]:
upload6_9.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data6/upload6_9.csv',index = False, header = False)

# data7

In [16]:
train = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data7/train.csv', index_col = None, header = None)
test = pd.read_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data7/test.csv', index_col = None, header = None)
data = pd.concat([train, test], axis = 0)
# col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8']
# data.columns = col_name

In [17]:
print(train.shape)
print(test.shape)

(231, 7)
(92, 7)


In [ ]:
data

,0,1,2,3,4,5,6
0,-2.3,0.568,NaN,NaN,3.17,0.150,0.27
1,-2.3,0.568,4.78,3.99,3.17,0.200,0.78
2,-2.3,0.568,4.78,NaN,3.17,0.225,1.18
3,-2.3,0.568,4.78,NaN,3.17,NaN,1.82
4,-2.3,0.568,4.78,3.99,NaN,NaN,3.76
...,...,...,...,...,...,...,...
87,-2.3,NaN,4.78,NaN,3.64,0.250,0.00
88,0.0,0.565,5.10,3.94,3.51,0.200,0.00
89,-2.3,0.562,NaN,4.95,NaN,NaN,0.00
90,0.0,0.530,4.78,3.75,3.15,0.175,0.00


In [ ]:
data_all = data.copy()
data_x = data_all.drop([6], axis = 1)
data_y = data_all[6] #predict
data_y = pd.DataFrame(data_y)

In [ ]:
from kuma_utils.preprocessing.imputer import LGBMImputer
lgbm_imtr = LGBMImputer(n_iter=500)
data_x_impute = lgbm_imtr.fit_transform(data_x)
data_x_impute = pd.DataFrame(data_x_impute)
data_x_impute

  0%|          | 0/6 [00:00<?, ?it/s]

,0,1,2,3,4,5
0,-2.3,5.680000e-01,4.170762,3.168781,3.170000,0.150
1,-2.3,5.680000e-01,4.780000,3.990000,3.170000,0.200
2,-2.3,5.680000e-01,4.780000,2.921560,3.170000,0.225
3,-2.3,5.680000e-01,4.780000,2.988294,3.170000,0.000
4,-2.3,5.680000e-01,4.780000,3.990000,2.989053,0.000
...,...,...,...,...,...,...
87,-2.3,2.861023e-08,4.780000,3.255859,3.640000,0.250
88,0.0,5.650000e-01,5.100000,3.940000,3.510000,0.200
89,-2.3,5.620000e-01,4.914128,4.950000,2.801976,0.000
90,0.0,5.300000e-01,4.780000,3.750000,3.150000,0.175


In [ ]:
X_train = data_x_impute.iloc[:train.shape[0]]
X_test = data_x_impute.iloc[-test.shape[0]:]
Y_train = data_y.iloc[:train.shape[0]]
Y_test = data_y.iloc[-test.shape[0]:]
scaler = StandardScaler()
train_x = scaler.fit_transform(X_train)
test_x = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(24, input_shape=(6,), activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation=None))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=200, batch_size=10)
ypred = model.predict(X_test)
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload7_12 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)
upload7_12.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data7/upload7_12.csv',index = False, header = False)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0, learning_rate=0.1, n_estimators=550, max_depth=4, min_child_weight=5, gamma=0.1) 
print(xgbr)
xgbr.fit(X_train, Y_train)
ypred = xgbr.predict(X_test)

XGBRegressor(gamma=0.1, max_depth=4, min_child_weight=5, n_estimators=550,
             verbosity=0)


In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.01,
    n_estimators=3000,
    objective='regression', 
    metric='mae'
)


print(model)
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

LGBMRegressor(learning_rate=0.01, metric='mae', n_estimators=3000,
              objective='regression')


In [ ]:
Y_test = pd.DataFrame(ypred)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
upload7_9 = pd.concat([X_test, Y_test], axis = 1, ignore_index=True)

In [ ]:
upload7_9.to_csv('/content/drive/MyDrive/fourth_year_class/big_data/hw3/data7/upload7_9.csv',index = False, header = False)